<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Training2/blob/main/web_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
HEADER = {
       'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
}


type_list = {
    "playing":"/movie_intheaters.html",
    "upcoming":"/movie_comingsoon.html",
    "thisweek":"/movie_thisweek.html"
}



# Now everything is ready,make each process as a fucnction

In [26]:

def getAllRelateMovieURL(movieURL):
  #check current movie is visted or not
  if(movieURL in movieLinkVisied): # not None ,movie is already visted
    return

  movieLinkVisied.add(movieURL) #set is now visting and 
  #get detail info with 
  reqDetail = requests.get(movieURL,headers=HEADER)
  detailSoup = BeautifulSoup(reqDetail.text)
  detailSoup = detailSoup.find("div",id="content_l")

  #getting the basic 
  # detailSoup.find("div",class_="movie_intro_info_r")
  info = detailSoup.find("div",class_="movie_intro_info_r")
  # print("Name",info.find("h1").text.strip())
  currentData["MovieName"] = info.find("h1").text.strip()

  #getting movie genre
  genreList = info.findAll("div",class_="level_name")
  genresList = []
  for genre in genreList:
    genresList.append(genre.find("a",class_="gabtn").text.strip())
  currentData["genre"] = ",".join(genresList)

  release = info.findAll("span")[0].text
  currentData["releaseDate"] = release.split("上映日期：")[1].strip()
  
  #getting movie intro
  intro = detailSoup.find("div",class_="gray_infobox_inner")
  introData = intro.find("span",id="story").text.strip()
  currentData["intro"] = introData

  if(detailSoup.findAll("div",class_="l_box have_arbox _c")[0].find("ul","trailer_list alist starlist") != None):
      actorList = detailSoup.findAll("div",class_="l_box have_arbox _c")[0].find("ul","trailer_list alist starlist").findAll("li")
      for actor in actorList:
        # currentActor = actor.find("a",class_="gabtn").find("div",class_="actor_inner").find("h2").text.strip()
        currentActorLink = actor.find("a",class_="gabtn")["href"]
        getActorMovieInfo(currentActorLink) # this action will append all none visited movie of current movie action had-----------------------------------------------------------------------------------------------------------------------------------------\n")

#Working
def getActorMovieInfo(actorURL):
  #check current Actor is already visted
  if(actorURL in actorLinkVisted): #Not None: is already visited
    return

  actorPage = requests.get(actorURL,headers=HEADER)
  actorSoup = BeautifulSoup(actorPage.text)

  actorLinkVisted.add(actorURL)

  #condition is there is any actor?
  if(actorSoup.find("ul",class_="trailer_list _slickfive") != None):
    for movie in actorSoup.find("ul",class_="trailer_list _slickfive").findAll("li"):
        url = movie.find("div",class_="_slickcontent").find("a",class_="gabtn")['href']
        if(url in movieLinkVisied): # not None ,movie is already visted
          continue
        movieUrlList.append(url)

In [27]:

def webCrawler(baseURL,movieType,pages=1):
  #need some error handle here

  for page in range(1,pages + 1):
    webURL = baseURL + "/"+ movieType +"?page=%s" %page

    #now sending the request
    movieReq = requests.get(webURL,headers=HEADER)
    currentPage = BeautifulSoup(movieReq.text)

    #check current page is empty/no movie there
    if(currentPage.find("div",class_="release_box").find("ul",class_="release_list").find("a",class_="btn_s_introduction") == None):
      return

    movieList = currentPage.find("div",class_="release_box").find("ul",class_="release_list").find("a",class_="btn_s_introduction")['href']

    movieData = currentPage.findAll("div",class_="release_info")
    for item in movieData:
        # get movie name
        # print(item.find("div",class_="release_movie_name").find("a",class_="gabtn").text.strip())

        #get movie detail URL,all detail 
        # print(item.find("div",class_="release_btn color_btnbox").find("a",class_="btn_s_introduction")["href"])
        detailURL = item.find("div",class_="release_btn color_btnbox").find("a",class_="btn_s_introduction")["href"]
        movieUrlList.append(detailURL)

In [36]:
movieUrlList = []
actorLinkVisted = set()
movieLinkVisied = set()
currentData = {}


webCrawler("https://movies.yahoo.com.tw",type_list["playing"],1)
webCrawler("https://movies.yahoo.com.tw",type_list["upcoming"],7)
webCrawler("https://movies.yahoo.com.tw",type_list["thisweek"],7)


In [30]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [37]:
#now will loop over the current list and add new item via its actor
import csv
#add to csv every 5 time
with open("./movie.csv","w") as movie:
  fieldnames = ["MovieName","genre","releaseDate","intro"]
  writer = csv.DictWriter(movie,fieldnames=fieldnames)
  writer.writeheader()
  for i in movieUrlList:
    getAllRelateMovieURL(i)
    writer.writerow(currentData)

In [51]:
for i in movieUrlList:
  print(i)

Streaming output truncated to the last 5000 lines.
https://movies.yahoo.com.tw/movieinfo_main/%E5%8D%81%E4%BA%8C%E5%80%8B%E6%83%B3%E6%AD%BB%E7%9A%84%E5%B0%91%E5%B9%B4-12-suicidal-teens-9518
https://movies.yahoo.com.tw/movieinfo_main/%E9%8A%80%E9%AD%822%E7%89%B9%E5%88%A5%E7%AF%87-%E4%B8%96%E7%95%8C%E5%A5%87%E5%A6%99%E9%8A%80%E9%AD%82%E9%86%AC-gintama-2-the-exceedingly-strange-gintama-chan-8761
https://movies.yahoo.com.tw/movieinfo_main/%E9%8A%80%E9%AD%822-%E8%A6%8F%E7%9F%A9%E6%98%AF%E7%82%BA%E4%BA%86%E8%A2%AB%E6%89%93%E7%A0%B4%E8%80%8C%E5%AD%98%E5%9C%A8%E7%9A%84-gintama-2-8271
https://movies.yahoo.com.tw/movieinfo_main/%E9%BD%8A%E6%9C%A8%E6%A5%A0%E9%9B%84%E7%9A%84%E7%81%BD%E9%9B%A3-psychic-kusuo-8030
https://movies.yahoo.com.tw/movieinfo_main/%E9%8A%80%E9%AD%82-gintama-live-action-the-movie-6948
https://movies.yahoo.com.tw/movieinfo_main/%E6%9A%97%E6%AE%BA%E6%95%99%E5%AE%A4-%E7%95%A2%E6%A5%AD%E7%AF%87-assassination-classroom-2-6342
https://movies.yahoo.com.tw/movieinfo_main/%E6%B0%B4%E6

In [48]:
import pandas as pd
datas = pd.read_csv("./movie.csv")
datas.drop_duplicates(keep="first",inplace=False)


,MovieName,genre,releaseDate,intro
0,路卡的夏天,"動畫,冒險",2021-08-06,皮克斯全新原創《路卡的夏天》，故事背景發生在美麗的義大利濱海小鎮里維埃拉，一位名叫路卡的海怪...
1,薄荷糖,劇情,2021-08-06,★韓國重量級名導李滄東「綠色三部曲」之一！ \r\n★勇奪「南韓奧斯卡」大鐘獎最佳影片、導演...
2,人聲,劇情,2021-08-06,一個焦慮的女人等著要跟她分手的男人可以出現，取回他的行李，心裡有著一絲絲盼望，也許可以有轉圜...
3,雙面薇若妮卡 經典數位修復,劇情,2021-08-06,★ 1991坎城影展最佳女主角、國際影評人費比西獎、人道主義精神獎\r\n★ 1992 金球...
4,合法伴侶,愛情,2021-08-06,★英國倫敦的異國追愛故事！無論是男女戀，還是男男戀？結婚才是合法，敢登記才是真愛\r\n★台...
...,...,...,...,...
12147,11月的孩子,劇情,2009-11-06,★德國紀實導演 克利斯蒂安施沃喬夫 寬恕歷史之作★德國影視展 觀眾票選最佳影片柏林圍牆倒塌，...
12148,七年之夜,"劇情,懸疑/驚悚",2018-04-27,★ 《七年之夜》改編韓國重量級作家丁柚井同名小說，聲勢直逼村上春樹，超越丹布朗《地獄》\r\...
12149,梵谷到我家,"愛情,喜劇",2021-05-07,瘋狂畫家穿越時空只為追尋真愛？\r\n窮到吃軟飯的畫家李奧答應幫廣告公司繪製一張惡搞梵谷的廣...
12150,終極殺陣：最後衝刺,"動作,喜劇",2007-08-24,超殺劇情，速度就是武器！大導演盧貝松最新監製作品，【終極殺陣】系列最終章，正宗經典【終極殺陣...
